In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

torch.set_num_threads(torch.get_num_threads())  # typically max threads
torch.set_num_interop_threads(torch.get_num_interop_threads())

In [4]:
class HairTypeCNN(nn.Module):
    """
    A small Convolutional Neural Network for identifying hair types
    from 200x200 color images (e.g., curly vs. straight).
    """

    def __init__(self):
        super().__init__()

        # Input: 3 x 200 x 200 hair image
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv and pooling:
        # 200 → 198 → pooled to 99
        flattened_size = 32 * 99 * 99

        self.fc1 = nn.Linear(flattened_size, 64)
        self.fc2 = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()   # binary hair-type output (0 or 1)

    def forward(self, x):

        x = self.relu(self.conv1(x))


        x = self.pool(x)


        x = x.view(x.size(0), -1)


        x = self.relu(self.fc1(x))
        x = self.fc2(x)


        return self.sigmoid(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HairTypeCNN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCELoss()


In [5]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
           Sigmoid-7                    [-1, 1]               0
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


In [6]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [7]:
train_dataset = ImageFolder(
    "C:/Users/mm3le/PycharmProjects/ml-zoomcamp/08-Neural-Networks-HW/data/train",
    transform=train_transforms
)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [8]:
validation_dataset = ImageFolder("C:/Users/mm3le/PycharmProjects/ml-zoomcamp/08-Neural-Networks-HW/data/test",
                                 transform=train_transforms)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.1798, Acc: 0.4875, Val Loss: 0.7692, Val Acc: 0.4876
Epoch 2/10, Loss: 0.1610, Acc: 0.4875, Val Loss: 0.7847, Val Acc: 0.4876
Epoch 3/10, Loss: 0.1376, Acc: 0.4888, Val Loss: 1.0123, Val Acc: 0.4876
Epoch 4/10, Loss: 0.1026, Acc: 0.4875, Val Loss: 0.8488, Val Acc: 0.4876
Epoch 5/10, Loss: 0.0456, Acc: 0.4888, Val Loss: 0.9437, Val Acc: 0.4876
Epoch 6/10, Loss: 0.0309, Acc: 0.4875, Val Loss: 0.9498, Val Acc: 0.4876
Epoch 7/10, Loss: 0.0231, Acc: 0.4888, Val Loss: 0.9089, Val Acc: 0.4925
Epoch 8/10, Loss: 0.0172, Acc: 0.4888, Val Loss: 0.9695, Val Acc: 0.4925
Epoch 9/10, Loss: 0.0138, Acc: 0.4900, Val Loss: 1.0046, Val Acc: 0.4925
Epoch 10/10, Loss: 0.0118, Acc: 0.4913, Val Loss: 1.0972, Val Acc: 0.4925


In [13]:
print(np.median(history['acc']), np.std(history['acc']))


0.48875 0.0011792476415070838


In [14]:
print(np.average(history['val_loss']))

0.9288506045566862 0.0024373032266499176


In [15]:
last5_avg = np.mean(history['val_acc'][-5:])
print(last5_avg)

0.4915422885572139
